In [1]:
from dotenv import load_dotenv
import os

# Load the environment variables from the .env file
load_dotenv()

# Access environment variables
api_key = os.getenv("OPENAI_KEY")

In [2]:
# extract data from the punk api
import requests
import pandas as pd

url = "https://api.punkapi.com/v2/beers"
beers = []
for i in range(1,10):
    params ={"per_page":80,
            "page":i}
    response = requests.get(url, params=params).json()
    if len(response) == 0:
        break
    else:
        beers.extend(response)


In [3]:
# now we have our data from the api we can vizualise it
df = pd.DataFrame(beers)
df.to_csv("beers.csv", index=False)
df.head(3)

,id,name,tagline,first_brewed,description,image_url,abv,ibu,target_fg,target_og,...,srm,ph,attenuation_level,volume,boil_volume,method,ingredients,food_pairing,brewers_tips,contributed_by
0,1,Buzz,A Real Bitter Experience.,09/2007,"A light, crisp and bitter IPA brewed with Engl...",https://images.punkapi.com/v2/keg.png,4.5,60.0,1010.0,1044.0,...,10.0,4.4,75.0,"{'value': 20, 'unit': 'litres'}","{'value': 25, 'unit': 'litres'}","{'mash_temp': [{'temp': {'value': 64, 'unit': ...","{'malt': [{'name': 'Maris Otter Extra Pale', '...","[Spicy chicken tikka masala, Grilled chicken q...",The earthy and floral aromas from the hops can...,Sam Mason <samjbmason>
1,2,Trashy Blonde,You Know You Shouldn't,04/2008,"A titillating, neurotic, peroxide punk of a Pa...",https://images.punkapi.com/v2/2.png,4.1,41.5,1010.0,1041.7,...,15.0,4.4,76.0,"{'value': 20, 'unit': 'litres'}","{'value': 25, 'unit': 'litres'}","{'mash_temp': [{'temp': {'value': 69, 'unit': ...","{'malt': [{'name': 'Maris Otter Extra Pale', '...","[Fresh crab with lemon, Garlic butter dipping ...",Be careful not to collect too much wort from t...,Sam Mason <samjbmason>
2,3,Berliner Weisse With Yuzu - B-Sides,Japanese Citrus Berliner Weisse.,11/2015,Japanese citrus fruit intensifies the sour nat...,https://images.punkapi.com/v2/keg.png,4.2,8.0,1007.0,1040.0,...,4.0,3.2,83.0,"{'value': 20, 'unit': 'litres'}","{'value': 25, 'unit': 'litres'}","{'mash_temp': [{'temp': {'value': 60, 'unit': ...","{'malt': [{'name': 'Propino Pale Malt', 'amoun...","[Smoked chicken wings, Miso ramen, Yuzu cheese...",Clean everything twice. All you want is the cl...,Sam Mason <samjbmason>


In [4]:
# we can see there are a number of columns we don't need and others that will need to be formatted 

In [5]:
# remove columns I don't think we'll need for recomendations (I may be wrong?)
df = df.drop(columns=["volume", "boil_volume", "method","brewers_tips", "attenuation_level", "target_fg", "target_og"])

# Get the ingredients people may be interested in - malts, hops or yeast (do they?)
df['malts'] = df['ingredients'].apply(lambda x: [i['name'] for i in x['malt']]).apply(lambda x: ", ".join(x))
df['hops'] = df['ingredients'].apply(lambda x: [i['name'] for i in x['hops']]).apply(lambda x: ", ".join(x))
df['yeast'] = df['ingredients'].apply(lambda x: x['yeast'])

# Create a column for all ingredients
df["ingredients"] = df["malts"] + " " + df["hops"] + " " + df["yeast"]

# Create a string from the recomendations list
df['food_pairing'] = df['food_pairing'].apply(lambda x: ", ".join(x))

# format the contributed_by column
df['contributed_by'] = df['contributed_by'].apply(lambda x: x.split(" <")[0])

df.head(3)

,id,name,tagline,first_brewed,description,image_url,abv,ibu,ebc,srm,ph,ingredients,food_pairing,contributed_by,malts,hops,yeast
0,1,Buzz,A Real Bitter Experience.,09/2007,"A light, crisp and bitter IPA brewed with Engl...",https://images.punkapi.com/v2/keg.png,4.5,60.0,20.0,10.0,4.4,"Maris Otter Extra Pale, Caramalt, Munich Fuggl...","Spicy chicken tikka masala, Grilled chicken qu...",Sam Mason,"Maris Otter Extra Pale, Caramalt, Munich","Fuggles, First Gold, Fuggles, First Gold, Cascade",Wyeast 1056 - American Ale™
1,2,Trashy Blonde,You Know You Shouldn't,04/2008,"A titillating, neurotic, peroxide punk of a Pa...",https://images.punkapi.com/v2/2.png,4.1,41.5,15.0,15.0,4.4,"Maris Otter Extra Pale, Caramalt, Munich Amari...","Fresh crab with lemon, Garlic butter dipping s...",Sam Mason,"Maris Otter Extra Pale, Caramalt, Munich","Amarillo, Simcoe, Amarillo, Motueka",Wyeast 1056 - American Ale™
2,3,Berliner Weisse With Yuzu - B-Sides,Japanese Citrus Berliner Weisse.,11/2015,Japanese citrus fruit intensifies the sour nat...,https://images.punkapi.com/v2/keg.png,4.2,8.0,8.0,4.0,3.2,"Propino Pale Malt, Wheat Malt, Propino Pale Ma...","Smoked chicken wings, Miso ramen, Yuzu cheesecake",Sam Mason,"Propino Pale Malt, Wheat Malt, Propino Pale Ma...",Bramling Cross,Wyeast 1056 - American Ale™


In [6]:
df.to_csv("clean_beer.csv", index=False)
df[:10].to_csv('clean_beer_test.csv', index=False)

In [15]:
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.agents.agent_types import AgentType
from langsmith import Client
from langchain.smith import RunEvalConfig, run_on_dataset


df = pd.read_csv("clean_beer.csv")

def get_chain():
    llm = ChatOpenAI(temperature=0, openai_api_key=api_key)
    agent_executor_kwargs = {
        "handle_parsing_errors": True,
    }
    agent = create_pandas_dataframe_agent(
        llm,
        df,
        agent_type=AgentType.OPENAI_FUNCTIONS,
        agent_executor_kwargs=agent_executor_kwargs,
        max_iterations=5
    )
    return agent

In [8]:
qa_chain = get_chain()

In [9]:
qa_chain('whats the average ABV?')

{'input': 'whats the average ABV?',
 'output': 'The average ABV (Alcohol By Volume) of the beers in the dataframe is approximately 7.91%.'}

In [12]:
qa_chain('which is the strongest beer?')

{'input': 'which is the strongest beer?',
 'output': 'The strongest beer in the dataframe is "The End Of History" with an ABV (Alcohol By Volume) of 55.0%.'}

In [19]:
answer = qa_chain('How many beers have an abv above 10%?')
print(answer['output'])

There are 63 beers in the dataframe with an ABV (Alcohol By Volume) above 10%.


In [21]:
answer = qa_chain('can you suggest a beer to go with my chicken wings?')
print(answer['output'])

Sure! I can help you with that. Can you please provide me with some more information about the chicken wings? Are they spicy, BBQ flavored, or something else?


In [22]:
answer = qa_chain('i like spicy chicken wings?')
print(answer['output'])

That's great! Spicy chicken wings are a delicious choice. Is there anything specific you would like to know or discuss about spicy chicken wings?


In [20]:
len(df[df["abv"] > 10])

63